In [88]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os

In [89]:
#Pull in crosswalk with nws.usgs, and nwm id and hefs flood thresholds in flow
sites = pd.read_csv('gage_id_nwm_feature_id_crosswalk.csv')
thresholds = pd.read_csv('threshold_data.csv')

In [90]:
#Creating a common key name for merging with the sites dataframe
thresholds['nws_station_id'] = thresholds['lid']

# Merge dataframes on the common column (nws_station_id)
nwm_nwps = pd.merge(sites, thresholds, on='nws_station_id', how='inner')  # You need to replace 'key' with the actual common column name

#Longitude needs to be converted to negative
nwm_nwps['longitude'] = nwm_nwps['longitude']*-1

#Drop NaN values. Without IDs we will not be able to pull data for them. Unlikley to be very many.
nwm_nwps.dropna(subset=['nws_station_id', 'gage_id', 'nwm_feature_id', 'crosswalk_source'], inplace=True)

#This line filters out the rows that are not Hawaii ot Puerto Rico Sites.
nwm_nwps = nwm_nwps.loc[(nwm_nwps['state_abbreviation'] ==  'HI')|(nwm_nwps['state_abbreviation'] ==  'PR')]

#Testing feature to shorten the number of sites for qicker testing
#nwm_nwps = nwm_nwps.head(50).copy()

In [91]:
# Function to fetch maximum flow from API for a given station ID
def get_max_flow(nwm_id):
    try:
        # API URL
        api_url = f'https://testing-api.water.noaa.gov/nwps/v1/reaches/{nwm_id}/streamflow?series=short_range'
        
        # Send a GET request to the API URL
        response = requests.get(api_url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Extract JSON data from the response
            json_data = response.json()
            
            # Extract the 'mediumRange' data from the JSON
            #medium_range_data = json_data.get('mediumRangeBlend', {})
            
            # Extract the 'mediumRangeBlend' data from the JSON
            medium_range_blend = json_data.get('shortRange', {})
            
            # Extract the 'series' data from 'mediumRangeBlend'
            series_data = medium_range_blend.get('series', {})
        
            # Extract the 'data' from 'series'
            data = series_data.get('data', [])
            
            # Convert the list of dictionaries into a pandas DataFrame
            df = pd.DataFrame(data)
            
            # Find the maximum flow
            max_flow = df['flow'].max()
            
            return max_flow
        else:
            print(f'Failed to fetch data from the API for station ID {nwm_id}. Status code:', response.status_code)
            return None
    except Exception as e:
        print(f'An error occurred while fetching data for station ID {nwm_id}:', str(e))
        return None

# Create a new column 'max_flow' in nwm_nwps
nwm_nwps['max_flow'] = nwm_nwps['nwm_feature_id'].apply(get_max_flow)

In [92]:
# Apply a lambda function to each row to find the nearest column
nwm_nwps['status'] = nwm_nwps.apply(lambda row: min(['action_flow_y', 'minor_flow_y', 'moderate_flow_y', 'major_flow_y'], 
                                                    key=lambda col: abs(row[col] - row['max_flow'])) 
                                     if any(row[col] < row['max_flow'] for col in ['action_flow_y', 'minor_flow_y', 'moderate_flow_y', 'major_flow_y'])
                                     else 'none', 
                                     axis=1)

In [93]:
#nwm_nwps.to_csv('delete.csv')

In [94]:
# Convert data to DataFrame
df = pd.DataFrame(nwm_nwps)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Load world and state shapefiles
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
us_states = gpd.read_file(os.path.join(os.getcwd(), 's_08mr23/s_08mr23.shp'))

# Filter data for Hawaii and Puerto Rico
gdf_hawaii = gdf[gdf['state_abbreviation'] == 'HI']
gdf_pr = gdf[gdf['state_abbreviation'] == 'PR']

# Plot for Hawaii including all islands
fig, ax = plt.subplots(figsize=(12, 8))
hawaii = us_states[us_states['STATE'] == 'HI']
hawaii.plot(ax=ax, color='lightgray', edgecolor='black')

# Plot points on the map based on status
colors = {'action': 'yellow', 'minor': 'orange', 'moderate': 'red', 'major': 'magenta'}
for status, color in colors.items():
    gdf_hawaii[gdf_hawaii['status'] == status].plot(ax=ax, color=color, markersize=100, label=status, alpha=0.7, marker='o')

ax.set_xlim([-161, -154])
ax.set_ylim([18.5, 23.0]) # Adjusted to include smaller islands
ax.set_title('NBM-Forced NWM Informed NWPS - Hawaii (All Islands)')
plt.legend()
plt.savefig('nbm_nwm_nwps_hawaii.png', dpi=300, bbox_inches='tight')
plt.close()

# Plot for Puerto Rico
fig, ax = plt.subplots(figsize=(12, 8))
pr = us_states[us_states['STATE'] == 'PR']
pr.plot(ax=ax, color='lightgray', edgecolor='black')

# Plot points on the map based on status
for status, color in colors.items():
    gdf_pr[gdf_pr['status'] == status].plot(ax=ax, color=color, markersize=100, label=status, alpha=0.7, marker='o')

ax.set_xlim([-67.5, -65.5])
ax.set_ylim([17.8, 18.6])
ax.set_title('NBM-Forced NWM Informed NWPS - Puerto Rico')
plt.legend()
plt.savefig('nbm_nwm_nwps_puerto_rico.png', dpi=300, bbox_inches='tight')
plt.close()


C:\Users\david.smith\.conda\envs\dev2\lib\site-packages\geopandas\plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
C:\Users\david.smith\.conda\envs\dev2\lib\site-packages\geopandas\plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
C:\Users\david.smith\.conda\envs\dev2\lib\site-packages\geopandas\plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
C:\Users\david.smith\.conda\envs\dev2\lib\site-packages\geopandas\plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
C:\Users\david.smith\.conda\envs\dev2\lib\site-packages\geopand